# Setup and Data

In [ ]:
# Imports
import pandas as pd
import pyreadstat
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt

# Parameters
start_year = 2019
end_year = 2022
base_year = 2019
top_n = 10
years=range(start_year,end_year+1)
factor = 1 # Factor in case of missing prices. 0 = ignore, 1 = assume no change in price

# Load folder names
folder_names_pathname='Data_clean/CEX_folder_names.csv'
folder_names_df = pd.read_csv(folder_names_pathname)

# CEX data folder
cex_data_folder='/Users/roykisluk/Downloads/Consumer_Expenditure_Survey/'

####################################################

# Functions

# Total consumption, monthly, across all products
def total_consumption_value(df): 
    total_consumption = 0.0
    for j in range(0, len(df)):
        total_consumption += df['omdan'][j]
    return total_consumption

# Function to keep only shared prodcodes
def keep_shared_prodcodes(df1, df2):
    shared_prodcodes = set(df1['prodcode']).intersection(set(df2['prodcode']))
    df1_shared = df1[df1['prodcode'].isin(shared_prodcodes)].reset_index(drop=True)
    df2_shared = df2[df2['prodcode'].isin(shared_prodcodes)].reset_index(drop=True)
    return df1_shared, df2_shared

# Weighting by product
def weighting(df):
    weights = pd.DataFrame(df['prodcode'].unique(), columns=['prodcode'])
    weights['weight'] = 0.0
    total_consumption = total_consumption_value(df)
    for j in range(0, len(weights)):
        weights.loc[j, 'weight'] = df[df['prodcode'] == weights.loc[j, 'prodcode']]['omdan'].sum() / total_consumption
    return weights

# Effective price paid, on average, per product
def average_price(df):
    average_prices = pd.DataFrame(df['prodcode'].unique(), columns=['prodcode'])
    average_prices['price'] = 0.0
    for j in range(0, len(average_prices)):
        average_prices.loc[j, 'price'] = (df[df['prodcode'] == average_prices.loc[j, 'prodcode']]['mehir'] / df[df['prodcode'] == average_prices.loc[j, 'prodcode']]['kamut']).mean()
    return average_prices

# Laspeyres index
def Laspeyres(df_base, df_current):
    # Create index dataframe
    index_df = pd.DataFrame(df_base['prodcode'].unique(), columns=['prodcode'])
    index_df['index'] = 0.0
    # Calculate weights and average prices
    weights = weighting(df_base)
    average_prices_base = average_price(df_base)
    average_prices_current = average_price(df_current)
    # Merge weights and average prices into index dataframe
    index_df = index_df.merge(weights, on='prodcode', how='left')
    index_df = index_df.merge(average_prices_base, on='prodcode', how='left', suffixes=('', '_base'))
    index_df = index_df.merge(average_prices_current, on='prodcode', how='left', suffixes=('_base', '_current'))
    # Calculate index
    total_index = 0.0
    # Missing prices counter
    missing_base_prices = 0
    missing_current_prices = 0
    for j in range(len(index_df)):
        # Check for missing prices
        price_current = index_df.loc[j, 'price_current']
        price_base = index_df.loc[j, 'price_base']
        if price_base == 0 or pd.isna(price_base) or np.isinf(price_base):
            index_df.loc[j, 'index'] = factor * 100
            missing_base_prices += 1
            print(f"prodcode {index_df.loc[j, 'prodcode']}: invalid price_base")
            continue
        if price_current == 0 or pd.isna(price_current) or np.isinf(price_current):
            index_df.loc[j, 'index'] = factor * 100
            missing_current_prices += 1
            print(f"prodcode {index_df.loc[j, 'prodcode']}: invalid price_current")
            continue
        # Calculate index for each product
        index_df.loc[j, 'index'] = (price_current / price_base) * 100
    # Calculate yearly price index
    for j in range(len(index_df)):
        weight = index_df.loc[j, 'weight']
        total_index += weight * index_df.loc[j, 'index']
    print(f"Missing base prices: {missing_base_prices}")
    print(f"Missing current prices: {missing_current_prices}")
    return index_df, total_index

def merge_to_secondary(df):
    df['prodcode_secondary'] = df['prodcode'].astype(str).str[:3]
    grouped = df.groupby('prodcode_secondary', group_keys=False).apply(
        lambda x: pd.Series({
            'price_index': np.average(x['index'], weights=x['weight']) if x['weight'].sum() > 0 else np.nan,
            'total_weight': x['weight'].sum()
        }),
        include_groups=False 
    ).reset_index()
    grouped.rename(columns={'prodcode_secondary': 'prodcode'}, inplace=True)
    grouped.rename(columns={'total_weight': 'weight'}, inplace=True)
    return grouped

def merge_to_primary(df):
    df['prodcode_primary'] = df['prodcode'].astype(str).str[:2]
    
    grouped = df.groupby('prodcode_primary', group_keys=False).apply(
        lambda x: pd.Series({
            'price_index': np.average(x['price_index'], weights=x['weight']) if x['weight'].sum() > 0 else np.nan,
            'total_weight': x['weight'].sum()
        }),
        include_groups=False
    ).reset_index()
    grouped.rename(columns={'prodcode_primary': 'prodcode'}, inplace=True)
    grouped.rename(columns={'total_weight': 'weight'}, inplace=True)
    return grouped



In [ ]:
# Load price data for each year
dfs_prices = {}
for year in years:
    subfolder = folder_names_df.loc[folder_names_df['Year'] == year, 'Folder_Name'].values[0]
    data_prices_pathname = f"{cex_data_folder}{subfolder}/{subfolder}datayoman.sas7bdat"
    df, meta = pyreadstat.read_sas7bdat(data_prices_pathname)
    df.columns = df.columns.str.lower()
    dfs_prices[year] = df

In [ ]:
# Filter observations with prodcode that starts with 3
for year in years:
    dfs_prices[year] = dfs_prices[year][dfs_prices[year]['prodcode'].astype(str).str.startswith('3')].reset_index(drop=True)

In [ ]:
# Calculate weights and price indexes
yearly_price_index = {}
df_price_index = {}
for year in years:
    df_base, df_current = keep_shared_prodcodes(dfs_prices[base_year], dfs_prices[year])
    df_price_index[year], yearly_price_index[year] = Laspeyres(df_base, df_current)

In [ ]:
# Combine all years into a single dataframe
combined_df = pd.concat(df_price_index.values(), keys=df_price_index.keys(), names=['Year', 'Index']).reset_index(level='Index', drop=True).reset_index()
combined_df = combined_df[['Year', 'prodcode', 'index', 'weight']]
print(combined_df)

In [ ]:
# Merge to secondary and primary categories
df_secondary = {}
df_primary = {}
for year in years:
    df_secondary[year] = merge_to_secondary(df_price_index[year])
    df_primary[year] = merge_to_primary(df_secondary[year])

# Combine secondary and primary categories into a single dataframe
combined_secondary_df = pd.concat(df_secondary.values(), keys=df_secondary.keys(), names=['Year', 'Index']).reset_index(level='Index', drop=True).reset_index()
combined_primary_df = pd.concat(df_primary.values(), keys=df_primary.keys(), names=['Year', 'Index']).reset_index(level='Index', drop=True).reset_index()

# Keep only the necessary columns
combined_secondary_df = combined_secondary_df[['Year', 'prodcode', 'price_index', 'weight']]
combined_primary_df = combined_primary_df[['Year', 'prodcode', 'price_index', 'weight']]

In [ ]:
# Load prodcode dictionary
prodcode_dict_pathname = 'Data_clean/prodcode_dictionary_c3-c399.csv'
prodcode_dict_df = pd.read_csv(prodcode_dict_pathname)

# Remove description column if it already exists
if 'description' in combined_secondary_df.columns:
    combined_secondary_df = combined_secondary_df.drop(columns=['description'])
if 'description' in combined_primary_df.columns:
    combined_primary_df = combined_primary_df.drop(columns=['description'])

# Convert prodcode to string in both dataframes before merging
prodcode_dict_df['prodcode'] = prodcode_dict_df['prodcode'].astype(str)
combined_secondary_df['prodcode'] = combined_secondary_df['prodcode'].astype(str)

# Merge descriptions into combined_secondary_df
combined_secondary_df = combined_secondary_df.merge(prodcode_dict_df, on='prodcode', how='left')

# Merge descriptions into combined_primary_df
combined_primary_df = combined_primary_df.merge(prodcode_dict_df, on='prodcode', how='left')

# Output

## Top Increases and Decreases

In [ ]:
# Display yearly price index per year in tabulate
print("Yearly Price Index:")
print(tabulate(yearly_price_index.items(), headers=["Year", "Price Index"], tablefmt="grid"))

In [24]:
from inflation_analysis import calculate_price_indexes

combined_df, combined_secondary_df, combined_primary_df = calculate_price_indexes()
df_top_secondary = {}
df_bottom_secondary = {}
df_top_primary = {}
df_bottom_primary = {}
for year in years:
    df_top_secondary[year] = combined_secondary_df[combined_secondary_df['Year'] == year].nlargest(top_n, 'price_index')
    df_bottom_secondary[year] = combined_secondary_df[combined_secondary_df['Year'] == year].nsmallest(top_n, 'price_index')
    df_top_primary[year] = combined_primary_df[combined_primary_df['Year'] == year].nlargest(top_n, 'price_index')
    df_bottom_primary[year] = combined_primary_df[combined_primary_df['Year'] == year].nsmallest(top_n, 'price_index')
    if year != base_year:
        print(f"Year: {year}")
        print("Top price indexes (Secondary):")
        print(tabulate(df_top_secondary[year], headers='keys', tablefmt='psql'))
        print("Bottom price indexes (Secondary):")
        print(tabulate(df_bottom_secondary[year], headers='keys', tablefmt='psql'))
        print("Top price indexes (Primary):")
        print(tabulate(df_top_primary[year], headers='keys', tablefmt='psql'))
        print("Bottom price indexes (Primary):")
        print(tabulate(df_bottom_primary[year], headers='keys', tablefmt='psql'))

prodcode 312090.0: invalid price_base
prodcode 383166.0: invalid price_base
prodcode 300418.0: invalid price_base
prodcode 343251.0: invalid price_base
prodcode 302133.0: invalid price_base
prodcode 309997.0: invalid price_base
prodcode 302034.0: invalid price_base
prodcode 302018.0: invalid price_base
prodcode 309088.0: invalid price_base
prodcode 303099.0: invalid price_base
prodcode 313189.0: invalid price_base
prodcode 398016.0: invalid price_base
prodcode 302224.0: invalid price_base
prodcode 302174.0: invalid price_base
prodcode 309344.0: invalid price_base
prodcode 312298.0: invalid price_base
prodcode 317198.0: invalid price_base
Missing base prices: 17
Missing current prices: 0
prodcode 312090.0: invalid price_base
prodcode 383166.0: invalid price_base
prodcode 300418.0: invalid price_base
prodcode 309153.0: invalid price_current
prodcode 343251.0: invalid price_base
prodcode 302133.0: invalid price_base
prodcode 309997.0: invalid price_base
prodcode 302034.0: invalid price_ba

In [ ]:
# Extract prodcodes for the last year
last_year = end_year

top_secondary_prodcodes = df_top_secondary[last_year]['prodcode'].unique()
bottom_secondary_prodcodes = df_bottom_secondary[last_year]['prodcode'].unique()
top_primary_prodcodes = df_top_primary[last_year]['prodcode'].unique()
bottom_primary_prodcodes = df_bottom_primary[last_year]['prodcode'].unique()

# Plot top secondary categories price indexes over time
plt.figure(figsize=(12, 8))
filtered_top_secondary_df = combined_secondary_df[combined_secondary_df['prodcode'].isin(top_secondary_prodcodes)]
for prodcode in filtered_top_secondary_df['prodcode'].unique():
    df_prodcode = filtered_top_secondary_df[filtered_top_secondary_df['prodcode'] == prodcode]
    description = df_prodcode['description'].iloc[0] if not df_prodcode['description'].isna().all() else f'Prodcode {prodcode}'
    plt.plot(df_prodcode['Year'], df_prodcode['price_index'], label=description)
plt.xlabel('Year')
plt.ylabel('Price Index')
plt.title('Top Secondary Categories Price Indexes Over Time')
plt.legend()
plt.grid(True)
plt.show()

# Plot bottom secondary categories price indexes over time
plt.figure(figsize=(12, 8))
filtered_bottom_secondary_df = combined_secondary_df[combined_secondary_df['prodcode'].isin(bottom_secondary_prodcodes)]
for prodcode in filtered_bottom_secondary_df['prodcode'].unique():
    df_prodcode = filtered_bottom_secondary_df[filtered_bottom_secondary_df['prodcode'] == prodcode]
    description = df_prodcode['description'].iloc[0] if not df_prodcode['description'].isna().all() else f'Prodcode {prodcode}'
    plt.plot(df_prodcode['Year'], df_prodcode['price_index'], label=description)
plt.xlabel('Year')
plt.ylabel('Price Index')
plt.title('Bottom Secondary Categories Price Indexes Over Time')
plt.legend()
plt.grid(True)
plt.show()

# Plot top primary categories price indexes over time
plt.figure(figsize=(12, 8))
filtered_top_primary_df = combined_primary_df[combined_primary_df['prodcode'].isin(top_primary_prodcodes)]
for prodcode in filtered_top_primary_df['prodcode'].unique():
    df_prodcode = filtered_top_primary_df[filtered_top_primary_df['prodcode'] == prodcode]
    description = df_prodcode['description'].iloc[0] if not df_prodcode['description'].isna().all() else f'Prodcode {prodcode}'
    plt.plot(df_prodcode['Year'], df_prodcode['price_index'], label=description)
plt.xlabel('Year')
plt.ylabel('Price Index')
plt.title('Top Primary Categories Price Indexes Over Time')
plt.legend()
plt.grid(True)
plt.show()